In [1]:
import sys
sys.path.append('../krpc-0.5.4')
import krpc

In [2]:
conn = krpc.connect(name='Sub-orbital flight')

In [ ]:
# 考虑到之后要操控助推器返程，卫星入轨，需要分别控制多个控制器，krpc确实可以做到这一点。
# 手工点火，然后分离
# 分离之后 1个有人核心，2个无人核心，一定要加入无人核心，否则不能控制飞行器。
conn.space_center.vessels

![Multiple vessels](../results/pictures/multiple_vessels_01.png "Multiple vessels")
下面组装两个火箭，然后分别用代码进行点火发射

~~注意一点的是，需要先启动点火，然后分离，然后才能分别控制阀门。
不是很理解这个逻辑
不过并不影响~~

In [ ]:
# 分别控制助推器不同推力
conn.space_center.vessels[1].control.throttle = 0.3
conn.space_center.vessels[2].control.throttle = 0.2

![Multiple vessels](../results/pictures/multiple_vessels_02.png "Multiple vessels")
这解决了一个关键问题，就是如何分别控制卫星入轨，助推器回收

In [ ]:
# 查看了另一个项目，是通过控制器名称来获得vessel的
# 和我的理解是一致的，不过用字典可能更好
def find_vessel_by_name(space_center, name):
    for v in space_center.vessels:
        if v.name == name:
            return v

最近的测试表明，并不需要控制器，Krpc有能力单独控制每一个组件(part), 同时，每个vessel需要激活才能具有control，才能activate_next_stage。

我在一个没有点火，手工分离的助推器上测试了以下代码，成功点火助推器。
```python
conn.space_center.active_vessel = conn.space_center.vessels[1]
conn.space_center.active_vessel.control.activate_next_stage()
```